In [17]:
import pandas as pd
import pymysql

# File paths for processed CSV files
file_paths = [
    'E:\\GUVI\\PROJECT 2 DATASPARK\\datasets\\Processed_Customers.csv',
    'E:\\GUVI\\PROJECT 2 DATASPARK\\datasets\\Processed_Exchange_rates.csv',
    'E:\\GUVI\\PROJECT 2 DATASPARK\\datasets\\Processed_Products.csv',
    'E:\\GUVI\\PROJECT 2 DATASPARK\\datasets\\Processed_Sales.csv',
    'E:\\GUVI\\PROJECT 2 DATASPARK\\datasets\\Processed_Stores.csv'
]

# MySQL connection parameters
host = '127.0.0.1'
user = 'root'
password = 'Angelica@14'
database = 'projectii'

# Connect to the MySQL database
connection = pymysql.connect(host=host, user=user, password=password, database=database)
cursor = connection.cursor()

# Map pandas dtypes to MySQL dtypes
dtype_mapping = {
    'object': 'TEXT',
    'int64': 'BIGINT',
    'int32': 'INT',
    'int8': 'TINYINT',
    'int16': 'SMALLINT',
    'float64': 'DOUBLE',
    'datetime64[ns]': 'DATE',
}

def create_table_from_df(df, table_name, cursor):
    # Drop the table if it already exists
    cursor.execute(f"DROP TABLE IF EXISTS `{table_name}`;")
    
    create_table_sql = f"CREATE TABLE `{table_name}` ("
    for column, dtype in df.dtypes.items():
        # Replace spaces with underscores and enclose in backticks
        column_name = column.replace(' ', '_')
        mysql_dtype = dtype_mapping.get(str(dtype), 'TEXT')
        create_table_sql += f"`{column_name}` {mysql_dtype}, "
    create_table_sql = create_table_sql.rstrip(', ') + ');'
    cursor.execute(create_table_sql)

def insert_data_into_table(df, table_name, cursor, connection):
    # Convert DataFrame to SQL insert statement
    cols = ', '.join([f"`{col.replace(' ', '_')}`" for col in df.columns])
    values = ', '.join(['%s'] * len(df.columns))
    insert_sql = f"INSERT INTO `{table_name}` ({cols}) VALUES ({values})"
    
    # Insert data in batches to avoid memory issues
    batch_size = 1000
    for start in range(0, len(df), batch_size):
        end = min(start + batch_size, len(df))
        batch = df.iloc[start:end]
        cursor.executemany(insert_sql, batch.values.tolist())
        connection.commit()

def create_and_insert_tables(file_paths, connection):
    cursor = connection.cursor()
    for file_path in file_paths:
        # Extract table name from file path
        table_name = file_path.split('\\')[-1].split('.')[0]
        
        # Read CSV into DataFrame
        df = pd.read_csv(file_path)
        
        # Convert date columns to datetime if they are not already
        date_columns = ['Order_Date', 'Delivery_Date', 'Open_Date', 'Birthday', 'Date']
        for col in date_columns:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col], errors='coerce')
        
        # Create table in MySQL
        create_table_from_df(df, table_name, cursor)
        print(f"Table {table_name} created.")
        
        # Insert data into the table
        insert_data_into_table(df, table_name, cursor, connection)
        print(f"Data inserted into {table_name}.")

# Call the function to create and insert data into MySQL tables
create_and_insert_tables(file_paths, connection)

# Close the connection
cursor.close()
connection.close()

print("Tables created and data inserted into MySQL database.")


Table Processed_Customers created.
Data inserted into Processed_Customers.
Table Processed_Exchange_rates created.
Data inserted into Processed_Exchange_rates.
Table Processed_Products created.
Data inserted into Processed_Products.
Table Processed_Sales created.
Data inserted into Processed_Sales.
Table Processed_Stores created.
Data inserted into Processed_Stores.
Tables created and data inserted into MySQL database.
